### Generate Cambdrige Data

This script leverages the CBECI API to download fresh Bitcoin energy consumption estimates from the [Cambridge Centre for Alternative Finance](https://ccaf.io/cbeci/index) and output the best guess annualized data for all energy price assumptions into a single CSV.

The motivation for this script is that the Cambridge Bitcoin Electricity Consumption Index assumes a constant fixed energy cost of $0.05/kWh, but in the face of rising energy prices there is desire to modify the output of the index to be more reflective of dynamic energy prices.

The output of this script is designed to be paired with EIA energy price data to allow for an evolving adjustment of what level of electricity spend the Bitcoin Network can support.

- Author: [Data Always](dataalways.substack.com)
- Last Modified: November 15, 2022

In [1]:
import numpy as np
import pandas as pd

In [2]:
"""
The analysis in this repository is only concerned with the CBECI
`BEST GUESS`. To avoid polution of the .csv we drop all the 
other columns. 
"""
def drop_standard_columns(dataframe):
    return dataframe.drop([
        'Timestamp', 
        'power MAX, GW', 
        'power MIN, GW', 
        'power GUESS, GW', 
        'annualised consumption MAX, TWh',
        'annualised consumption MIN, TWh'
        ], 
        axis= 1
    )

In [3]:
# Create the base dataframe on which to merge others by looking at the $0.01 data
df = pd.read_csv(
    "https://ccaf.io/cbeci/api/v1.2.0/download/data?price=0.01", 
    skiprows=1
)

# Drop the unwanted columns
df = drop_standard_columns(df)

# Rename the columns
df.rename(
    columns={
        'Date and Time': 'date', 
        'annualised consumption GUESS, TWh': 0.01
    }, 
    inplace=True
)

In [4]:
# Download the data for all prices and then merge them into one dataframe
for elec_cost in np.linspace(0.02, 0.19, 18):
    elec_str = "{:.2f}".format(elec_cost)
    
    tmp_df = pd.read_csv(
        f"https://ccaf.io/cbeci/api/v1.2.0/download/data?price={elec_str}", 
        skiprows=1
    )
    
    tmp_df = drop_standard_columns(tmp_df)
    
    tmp_df.rename(
        columns={
            'Date and Time': 'date', 
            'annualised consumption GUESS, TWh': elec_str
        }, 
        inplace=True
    )

    # Merge the data into the main data frame
    df = df.merge(
        tmp_df, 
        how="inner", 
        on="date"
    )

In [5]:
df.tail()

,date,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19
4790,2023-08-29T00:00:00,129.523169,129.523169,127.341411,123.666241,115.938646,107.408467,102.591809,95.922067,92.708035,85.029825,80.021469,77.424392,76.559644,73.129,73.10072,71.138983,74.596943,76.57994,62.27301
4791,2023-08-30T00:00:00,127.330200,127.330200,125.185381,121.240296,113.221264,105.277513,100.107110,93.794522,90.174675,82.882684,78.159315,75.859683,75.974231,73.129,73.10072,71.138983,74.596943,76.57994,62.27301
4792,2023-08-31T00:00:00,127.708846,127.708846,125.543870,121.430948,113.011017,105.516554,99.923799,93.925872,89.678160,82.564365,78.041068,75.800501,75.815783,73.129,73.10072,71.138983,74.596943,76.57994,62.27301
4793,2023-09-01T00:00:00,129.445696,129.445696,127.225459,122.826785,114.368781,107.140839,101.012243,95.351921,90.128054,83.532967,79.021121,76.625881,75.974231,73.129,73.10072,71.138983,74.596943,76.57994,62.27301
4794,2023-09-02T00:00:00,131.042413,131.042413,128.769103,124.086780,115.599619,108.649687,101.986909,96.866777,90.470928,84.408969,79.913578,77.365457,75.974231,73.129,73.10072,71.138983,74.596943,76.57994,62.27301


In [6]:
# Export the data to a .csv file
df.to_csv(
    './datasets/cbeci-annualized-consumption.csv', 
    index = False
)